In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import plotnine as p9
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# To analyze pA and rM soRNA-seq.
# Output: volcano

In [12]:
# solo rM-seq, yg samples.
df_rM_GV = pd.read_csv("../raw/RNAseq/dis3l2_cKO_vs_ctr_ERCC_rM.csv",skiprows=1, 
                    names=['gene_id', 'baseMean','log2FC','lfcSE','stat','p','padj'])
df_rM_GV
df_rM_GV_up = df_rM_GV[(df_rM_GV['log2FC']>1)&(df_rM_GV['padj']<0.01)]
df_rM_GV_dn = df_rM_GV[(df_rM_GV['log2FC']<-1)&(df_rM_GV['padj']<0.01)]
df_rM_GV_un = df_rM_GV[(-df_rM_GV['gene_id'].isin(df_rM_GV_up['gene_id'])) & (-df_rM_GV['gene_id'].isin(df_rM_GV_dn['gene_id']))]
df_rM_GV_up #6914
df_rM_GV_dn #444


,gene_id,baseMean,log2FC,lfcSE,stat,p,padj
70,ENSMUSG00000000531,71.291872,-2.176036,0.430850,-5.050567,4.405005e-07,2.030215e-06
91,ENSMUSG00000000701,1324.783583,-1.911844,0.196534,-9.727820,2.294567e-22,4.242965e-21
210,ENSMUSG00000001494,71.935853,-1.948397,0.425819,-4.575648,4.747476e-06,1.902907e-05
245,ENSMUSG00000001739,314.104045,-2.551220,0.310156,-8.225593,1.942267e-16,2.346970e-15
335,ENSMUSG00000002289,8.291995,-3.363714,1.106514,-3.039921,2.366406e-03,5.912143e-03
...,...,...,...,...,...,...,...
21093,ENSMUSG00000117235,12.402385,-3.236741,0.770712,-4.199678,2.672946e-05,9.575480e-05
21198,ENSMUSG00000117734,38.002165,-1.651568,0.561970,-2.938890,3.293902e-03,7.977618e-03
21249,ENSMUSG00000117972,3.034763,-4.705819,1.401347,-3.358069,7.848904e-04,2.160698e-03
21298,ENSMUSG00000118180,234.193775,-1.636658,0.245647,-6.662633,2.689650e-11,2.015917e-10


In [13]:
# pA-seq based on pA all GV and MII.
df_pA_GV = pd.read_csv("../raw/RNAseq/dis3l2_cKO_vs_ctr_ERCC_pA.csv",skiprows=1, 
                    names=['gene_id', 'baseMean','log2FC','lfcSE','stat','p','padj'])
df_pA_GV
df_pA_GV_up = df_pA_GV[(df_pA_GV['log2FC']>1)&(df_pA_GV['padj']<0.01)]
df_pA_GV_dn = df_pA_GV[(df_pA_GV['log2FC']<-1)&(df_pA_GV['padj']<0.01)]
df_pA_GV_un = df_pA_GV[(-df_pA_GV['gene_id'].isin(df_pA_GV_up['gene_id'])) & (-df_pA_GV['gene_id'].isin(df_pA_GV_dn['gene_id']))]

df_pA_GV_dn #1258
df_pA_GV_up #850



,gene_id,baseMean,log2FC,lfcSE,stat,p,padj
23,ENSMUSG00000000197,38.722449,1.778252,0.471871,3.768516,1.642209e-04,1.398827e-03
46,ENSMUSG00000000340,138.891097,1.373351,0.392972,3.494781,4.744513e-04,3.243914e-03
58,ENSMUSG00000000416,221.483897,1.374745,0.287225,4.786306,1.698793e-06,3.203598e-05
72,ENSMUSG00000000555,36.133684,1.912737,0.558407,3.425343,6.140236e-04,4.015595e-03
73,ENSMUSG00000000560,9.534302,5.263147,1.099627,4.786302,1.698820e-06,3.203598e-05
...,...,...,...,...,...,...,...
18594,ENSMUSG00000118127,6.938304,2.867821,0.841989,3.406006,6.592085e-04,4.245151e-03
18597,ENSMUSG00000118134,15.406249,2.611566,0.531809,4.910722,9.074158e-07,1.888861e-05
18601,ENSMUSG00000118189,4.439088,3.422529,0.998163,3.428826,6.061970e-04,3.973670e-03
18619,ENSMUSG00000118296,2.255583,4.114760,1.169087,3.519634,4.321424e-04,3.032199e-03


In [15]:
# done
# this cell is to make valcono plot of each df above
fold_sig = 1
padj_sig = 0.01
# define df and name here:
df_use = df_pA_GV.copy()
name = 'df_pA_GV'
# remove ERCC
df_use=df_use[df_use['gene_id'].str.startswith("ENSMU")]
# fill padj na =1
df_use=df_use.fillna(1)
# transfer all genes padj as -log10(p)
all_genes = df_use.copy()
all_genes['padj'] = -log10(all_genes['padj'])

# plot all data as gray background:
# rename colnames for long
all_genes.columns=['ID','basemean','log2FC','lfcSE','stat','pvalue','padj']
df_all_genes_long = all_genes[['ID','log2FC','padj']]
# define up and down
df_all_genes_long.loc[df_all_genes_long['log2FC']>fold_sig,'change'] = 'up'
df_all_genes_long.loc[df_all_genes_long['log2FC']<-fold_sig,'change'] = 'dn'

# set margin values all values > margin as margin
margin = 10
df_margin = df_all_genes_long.copy()
df_margin.loc[df_margin['log2FC']>= margin, 'log2FC']  = margin
df_margin.loc[df_margin['log2FC']<= - margin, 'log2FC']  = -margin
df_margin.loc[df_margin['padj']>= margin, 'padj']  = margin
df_margin.loc[df_margin['padj']<= - margin, 'padj']  = -margin
df_margin = df_margin.loc[(df_margin['log2FC']== margin)|
                          (df_margin['log2FC']== -margin) |
                          (df_margin['padj']== margin)|
                          (df_margin['padj']== -margin)]
# get the margin df:
df_margin
# get the all gene value for background plot:
df_all_genes_long

# pick genes having significant (<0.01) in at least ctr or cKO, which is log10p > 2
df_long_sig_group1 = df_all_genes_long[df_all_genes_long['padj']>-log10(padj_sig)]
                                        
plot = (
p9.ggplot()
    + p9.geom_point(data=df_all_genes_long,
           mapping=p9.aes(x='log2FC', y='padj'),color='gray',alpha=0.01)
    + p9.geom_point(data=df_long_sig_group1,
           mapping=p9.aes(x='log2FC', y='padj', color='change'),alpha=0.05)
    + p9.geom_point(data=df_margin,
           mapping=p9.aes(x='log2FC', y='padj', color = 'change'),shape='x')
    + p9.scale_color_manual(breaks = 'change',
                           values = ['blue','red'])
    + p9.theme(void)
    + p9.theme(axis_text_x = p9.element_text(angle=90))
    + p9.xlim(-margin,margin)
    + p9.ylim(0,margin)
    + p9.coord_fixed(ratio=2)
    + p9.geom_abline(intercept=0, slope=0, linetype = "dashed", color='gray')
    + p9.geom_vline(xintercept=0, linetype = "dashed", color='gray')
    + p9.ggtitle(name)
    + p9.ylab('-log10(padj)')
)
plot
plot.save("../results/pAvsrM_" + name + "_volcano.tiff", height=2, width=3,dpi=600)


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 3 x 2 in image.
  from_inches(height, units), units), PlotnineWarning)
/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggpl